In [1]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from boostvqe.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q3l/sgd_10q_3l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)
nq = config["nqubits"]
losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")


# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=nq, delta=0.5)
circuit = build_circuit(nq, config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(nq)
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)



[Qibo 0.2.9|INFO|2024-06-11 19:58:52]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0
[Qibo 0.2.9|INFO|2024-06-11 19:58:52]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy


In [2]:
from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,nq) ]
#b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(nq),b_list)]))
from copy import deepcopy
step = 0.12
u2_circ = deepcopy(d).circuit( -step * (np.sqrt(5) - 1) / 2 ) 
u4_circ = deepcopy(d).circuit( step * (np.sqrt(5) + 1) / 2 )
u6_circ = deepcopy(d).circuit( -step  ) 

from boostvqe.compiling_XXZ import *
vqe_circ = vqe.circuit
vqe_circ_inv = vqe_circ.invert()
nmb_ts_steps = 1
u1_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (np.sqrt(5) - 1) / 2,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 
u3_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t= step ,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 
u5_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (3 - np.sqrt(5)) / 2,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 

     
circ_boost = (  u6_circ + u5_circ + u4_circ + u3_circ + u2_circ +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.749515449118832

In [3]:
from qibo import symbols, hamiltonians
from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,nq) ]
#b_list = np.random.rand(nq)
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(nq),b_list)]))

step = 0.12

from boostvqe.models.dbi.group_commutator_iteration_transpiler import *
eo_d = EvolutionOracle(d,mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation,name = "D(linear)")

hamiltonian.delta = 0.5
from boostvqe.models.dbi.double_bracket_evolution_oracles import *
# quick hack how to use the method function hamiltonians.XXZ to get a simulable oracle
eo_xxz = XXZ_EvolutionOracle(hamiltonian,mode_evolution_oracle=EvolutionOracleType.numerical, steps = 1, order = 2)
eo_xxz.mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation
hamiltonian.dense = hamiltonian
# prescribe nmb of steps
eo_d.please_use_prescribed_nmb_ts_steps = 1
eo_xxz.please_use_prescribed_nmb_ts_steps = 1

In [4]:

diff_norms = []
diff_norms_2nd_order = []
times = np.linspace(0,.3,6)
a = lambda t_duration: nqubit_XXZ_decomposition(
            nqubits=nq,t=t_duration,delta=h_xxz.delta,steps=3,order=1)
b = lambda t_duration: nqubit_XXZ_decomposition(
            nqubits=nq,t=t_duration,delta=h_xxz.delta,steps=3,order=2)


In [5]:

gci  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=
                            FrameShiftedEvolutionOracle(eo_xxz, before_circuit=vqe.circuit.invert(), 
                                                        after_circuit=vqe.circuit,name="shifting by vqe"), 
                                                        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)


In [6]:
from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,nq) ]
#b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(nq),b_list)]))
from copy import deepcopy
step = 0.12
u2_circ = deepcopy(d).circuit( -step * (np.sqrt(5) - 1) / 2 ) 
u4_circ = deepcopy(d).circuit( step * (np.sqrt(5) + 1) / 2 )
u6_circ = deepcopy(d).circuit( -step  ) 

from boostvqe.compiling_XXZ import *
vqe_circ = vqe.circuit
vqe_circ_inv = vqe_circ.invert()
nmb_ts_steps = 1

u3_circ = vqe_circ +eo_xxz.circuit( step ,steps = 1, order =2 )+vqe_circ_inv
u5_circ = vqe_circ+ eo_xxz.circuit(-step * (3 - np.sqrt(5)) / 2 ,steps = 1, order =2 )+vqe_circ_inv
     
circ_boost = (  u6_circ + u5_circ + u4_circ + u3_circ + u2_circ +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.749515449118832

In [7]:
fsoe  = FrameShiftedEvolutionOracle(eo_xxz, before_circuit=vqe.circuit.invert(), 
                                                        after_circuit=vqe.circuit,name="shifting by vqe")

In [8]:
c = fsoe.circuit(0.1)

from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,nq) ]
#b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(nq),b_list)]))
from copy import deepcopy
step = 0.12
u2_circ = eo_d.circuit(-step * (np.sqrt(5) - 1) / 2) #deepcopy(d).circuit( -step * (np.sqrt(5) - 1) / 2 ) 
u4_circ = eo_d.circuit( step * (np.sqrt(5) + 1) / 2 )
u6_circ = eo_d.circuit( -step  ) 

from boostvqe.compiling_XXZ import *
vqe_circ = vqe.circuit
vqe_circ_inv = vqe_circ.invert()
nmb_ts_steps = 1

u3_circ = fsoe.circuit(step)
u5_circ = fsoe.circuit(-step * (3 - np.sqrt(5)) / 2  )
     
circ_boost = (  u6_circ + u5_circ + u4_circ + u3_circ + u2_circ +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 


-14.749515449118832

In [9]:

gci  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=fsoe, 
        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)

    
circ_boost = (  gci.group_commutator(step**2, eo_d)["forwards"] +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.749515449118832

In [10]:

gci  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=fsoe, 
        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)

    
circ_boost = (  gci.group_commutator(step**2, eo_d)["forwards"] +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.749515449118832

In [11]:
eo1 = eo_d
eo2 = fsoe
s_step = step
query_list_forward = [
                deepcopy(eo1).circuit(-s_step * (np.sqrt(5) - 1) / 2),
                deepcopy(eo2).circuit(s_step),
                deepcopy(eo1).circuit(s_step * (np.sqrt(5) + 1) / 2),
                deepcopy(eo2).circuit(-s_step * (3 - np.sqrt(5)) / 2),
                deepcopy(eo1).circuit(-s_step)            
            ]
query_list_backward = [ Circuit.invert(c) for c in query_list_forward[::-1]]
dict_circuit = {
                "forwards": reduce(Circuit.__add__, query_list_forward[::-1]),
                "backwards": reduce(Circuit.__add__, query_list_backward[::-1]),
            }



hamiltonian.expectation( (reduce(Circuit.__add__, query_list_forward[::-1])
               +vqe_circ)().state() )

-14.749515449118832

In [12]:
hamiltonian.expectation( (reduce(Circuit.__add__, [u1_circ,u2_circ,u3_circ,u4_circ,u5_circ,u6_circ][::-1])
               +vqe_circ)().state() )

-14.75006340353596

In [13]:
circ_boost =  ( reduce(Circuit.__add__, [u1_circ,u2_circ,u3_circ,u4_circ,u5_circ,u6_circ][::-1])
               +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.75006340353596

In [14]:
circ_boost =  ( eo1.circuit(-s_step) + 
               eo2.circuit(-s_step * (3 - np.sqrt(5)) / 2)+
               eo1.circuit(s_step * (np.sqrt(5) + 1) / 2)+
               eo2.circuit(s_step)+
               eo1.circuit(-s_step) 
               +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-14.7136267945728

In [21]:
gci(step**2, eo_d)
hamiltonian.expectation(gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit()().state())

-14.809004909737844

In [20]:
a = gci.iterated_hamiltonian_evolution_oracle.after_circuit
b = gci.iterated_hamiltonian_evolution_oracle.base_evolution_oracle.after_circuit
hamiltonian.expectation( (a+vqe.circuit)().state())

-14.503182773819251

In [17]:
print(target_energy)
print(ene1)

-15.276131122065795
-14.560174594055583


In [18]:
dm = d.dense.matrix

[Qibo 0.2.9|WARNING|2024-06-11 19:58:57]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


KeyboardInterrupt: 

In [ ]:
from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.single_commutator,
)


In [ ]:
dbi.h.expectation(zero_state_t)

-14.560174594055578

In [ ]:
dbi.h.nqubits

NameError: name 'dbi' is not defined

In [ ]:

for t in [0.01]:
    dbi(t)
    print(dbi.h.expectation(zero_state_t))
    dbi.h = deepcopy(h_vqe_ham)

-14.782805005074469


In [ ]:
def u_gcr(h,d,t):
    ud =  d.exp(t)
    u = h.exp(t)
    return     ud.conj().T @ u @ud


In [ ]:
h_vqe_ham.expectation(u_gcr(h_vqe_ham,d,np.sqrt(0.01))@zero_state_t)

-1.4827471249442334

In [ ]:
h_vqe_ham.expectation(u_gcr(h_vqe_ham,d,np.sqrt(0.01))@zero_state_t)

In [ ]:

# DBQA stuff
t0 = time.time()
print("Rotating with VQE")
new_hamiltonian_matrix = rotate_h_with_vqe(hamiltonian=hamiltonian, vqe=vqe)
new_hamiltonian = hamiltonians.Hamiltonian(
    nq, matrix=new_hamiltonian_matrix
)
print(time.time() - t0)


In [ ]:
t = 0.01
s = np.sqrt(t)

In [ ]:
target_energy

-15.276131122065795

In [ ]:
ene1

In [ ]:
from qibo import hamiltonians
import numpy as np
from boostvqe.compiling_XXZ import *

steps = 1
delta=0.5


h_xxz = hamiltonians.XXZ(nqubits=nq, delta = delta)
u = h_xxz.exp(t)


In [ ]:
from qibo import symbols, hamiltonians
d = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(nq)]))
circ_d = d.circuit(s)
ud = circ_d.unitary()

In [ ]:
from qibo import symbols, hamiltonians
dtest = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(nq)])).dense
ud =  dtest.exp(-0.1)

In [ ]:
u2 = h_xxz.exp(t*np.sqrt(5))

In [ ]:
print(h_xxz.expectation(ud.conj().T@u2@ud@u2@u_gcr@ vqe_state))

In [ ]:
circ = nqubit_XXZ_decomposition(nqubits=nq,t=t,delta=delta,steps=steps)
v = circ.unitary()



In [ ]:
u = v@v

In [ ]:
circ = nqubit_XXZ_decomposition(nqubits=nq,t=t * np.sqrt(5),delta=delta,steps=steps)
v2 = circ.unitary()

In [ ]:
def try_to_speed_up_GC3( v,v2,d):
    return  (
    v@ v2.conj().T
    @ self.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, d)
    @ v
    @ self.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, d)
    @ v.conj().T @  v.conj().T @v.conj().T @ v2.conj().T
    @ self.backend.calculate_matrix_exp(-step, d)
)


In [ ]:
from qibo import symbols, hamiltonians
d = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(nq)])).dense

In [ ]:
from qibo.models.dbi.utils import *

pauli_operator_dict = generate_pauli_operator_dict(nq, 1)
pauli_operators = list(pauli_operator_dict.values())
d_coef_pauli = np.ones(nq)
d_coef_pauli = d_coef_pauli + np.random.rand(len(d_coef_pauli))
d_pauli = sum([d_coef_pauli[i] * pauli_operators[i] for i in range(len(d_coef_pauli))])


In [ ]:
np.linalg.norm(d_pauli - d)

In [ ]:
q = try_to_speed_up_GC3(v,v2,dbi.)

In [ ]:
def expose_gci3(self, step)
    return = (
    self.h.exp(-step * (np.sqrt(5) - 1) / 2)
    @ self.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, d)
    @ self.h.exp(step)
    @ self.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, d)
    @ self.h.exp(-step * (3 - np.sqrt(5)) / 2)
    @ self.backend.calculate_matrix_exp(-step, d)
)



In [ ]:
u =expose_gci3(dbi,np.sqrt(t))


In [ ]:

dbi = DoubleBracketIteration(
    hamiltonian=new_hamiltonian,
    mode=DoubleBracketGeneratorType.group_commutator_3,
)

zero_state_t = np.transpose([zero_state])
energy_h0 = float(dbi.h.expectation(np.array(zero_state_t)))
fluctuations_h0 = float(dbi.h.energy_fluctuation(zero_state_t))


In [ ]:
dbi(0.1)

In [ ]:
d = dbi.diagonal_h_matrix
dbi.backend.calculate_matrix_exp(-0.1,d)

In [ ]:
dbi.backend.calculate_matrix_exp(-0.1,dbi.h.matrix)

In [ ]:
new_hamiltonian.exp(0.1)

In [ ]:
print(fluctuations_h0)
print(energy_h0)

In [ ]:
dbi.loss(0.01)

In [ ]:

print("Applying DBI steps")

print(time.time() - t0)
print(
    f"\nReached accuracy before DBI at iter {target_epoch}: {np.abs(target_energy - ene1)}"
)
print(
    f"Reached accuracy after DBI at iter {target_epoch}: {np.abs(target_energy - dbi_energies[-1])}"
)
print(
    f"Reached accuracy in the end of VQE long training: {np.abs(target_energy - losses[-1])}"
)
